In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import os

### Dataset Generation

In [2]:
num_samples = 40
np.random.seed(45) 
    
# Generate data
x1 = np.random.uniform(-20, 20, num_samples)
f_x = 100*x1 + 1
eps = np.random.randn(num_samples)
y1 = f_x + eps

np.random.seed(45)
num_samples = 40
    
# Generate data
x2 = np.random.uniform(-1, 1, num_samples)
f_x = 3*x2 + 4
eps = np.random.randn(num_samples)
y2 = f_x + eps

In [ ]:
def mse_loss(X,y, theta):
    pred = X @ theta
    return 0.5 * torch.mean((y-pred)**2)

In [ ]:
def comp_grad(X, y, theta):
    n = X.shape[0]
    grad = -(X.T @ (y- X @ theta))/n
    return grad

In [7]:
def fullgd(X, y, theta_star, lr=0.0001, eps=0.001, max_epoch=200):
    theta = torch.zeros(X.shape[1])
    losses, path = [],[]
    steps = 0

    for epoch in range(max_epoch):
        loss = mse_loss(X,y,theta)
        losses.append(loss.item())
        path.append(theta.clone().numpy())

        grad = comp_grad(X,y,theta)
        theta -= lr*grad

        steps += 1

        if torch.norm(theta - theta_star) < eps:
            break
    
    return theta, losses, np.array(path), steps

In [8]:
def sgd(X, y, theta_star, lr=0.0001, eps=0.001, max_epoch=200):
    theta = torch.zeros(X.shape[1])
    losses, path = [],[]
    steps = 0
    n = X.shape[0]

    for epoch in range(max_epoch):
        idx = torch.randperm(n)

        for i in idx:
            xi = X[i].unsqueeze(0)
            yi = y[i].unsqueeze(0)

            grad = comp_grad(xi, yi, theta)
            theta -= lr * grad

            path.append(theta.clone().numpy())
            steps += 1
            if torch.norm(theta - theta_star) < eps:
                losses.append(mse_loss(X, y, theta).item())
                return theta, losses, np.array(path), steps

        losses.append(mse_loss(X, y, theta).item())
    return theta, losses, np.array(path), steps